In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import prince  # Necesario para el análisis de correspondencias

# Cargar los datos
nombres_csv = {1,2,3,4,468,479,4003,4151,4160,4173}
pca_pesos={}
afc_pesos={}

for nombre_csv in nombres_csv:
    data = pd.read_csv(f'Data1_procesados/{nombre_csv}.csv')

    # Definir la fecha límite y filtrar el DataFrame
    fecha_limite = '2024-01-07'
    df_7days = data[data['Fecha'] <= fecha_limite].copy()

    # Normalizar los datos para PCA (sin 'Fecha' y 'Hora')
    scaler = StandardScaler()
    continue_data = df_7days[['period eating', 'period other', 'period resting', 'period rumination']]
    continue_data_scaled = scaler.fit_transform(continue_data)

    # Aplicar PCA para análisis de actividad
    pca = PCA(n_components=2)
    components = pca.fit_transform(continue_data_scaled)

    # Crear un DataFrame para los componentes principales
    loadings = pca.components_.T
    loadings_continue_data = pd.DataFrame(data=loadings, index=continue_data.columns, columns=['Principal Component 1', 'Principal Component 2'])

    # Calcular los pesos y el nivel de actividad basado en PCA
    pca_weights = (loadings_continue_data['Principal Component 1'] + loadings_continue_data['Principal Component 2']) / 2
    pca_pesos[nombre_csv]=pca_weights
    activity_level = continue_data.dot(pca_weights)
    # Añadir el nivel de actividad basado en PCA al DataFrame filtrado
    df_7days['nivel_actividad_1'] = activity_level

    # Calcular y añadir el nivel de actividad (PCA) al DataFrame original `data`
    data_continue = data[['period eating', 'period other', 'period resting', 'period rumination']]
    data['nivle_actividad_2'] = data_continue.dot(pca_weights)

    # Paso adicional: Aplicar AFC en columnas de conteo
    # Seleccionar solo las columnas de conteo para AFC
    count_data = df_7days[['period eating_count', 'period other_count', 'period resting_count', 'period rumination_count']]

    # Aplicar AFC
    ca = prince.CA(n_components=2, n_iter=10)
    ca = ca.fit(count_data)

    # Obtener coordenadas de columnas (variables) para AFC
    col_coords = ca.column_coordinates(count_data)

    # Calcular los pesos de AFC para cada variable
    afc_weights = (col_coords[0] + col_coords[1]) / 2
    afc_weights.index = count_data.columns  # Asegurarse de que los índices coincidan con las columnas originales
    afc_pesos[nombre_csv]=afc_weights
    # Calcular el nivel de actividad basado en AFC
    activity_level_2 = count_data.dot(afc_weights)
    df_7days['activity_level_2'] = activity_level_2

    # Calcular y añadir el nivel de actividad (AFC) al DataFrame original `data`
    data_count = data[['period eating_count', 'period other_count', 'period resting_count', 'period rumination_count']]
    data['activity_level_2'] = data_count.dot(afc_weights)

    data.to_csv(f'Data1_actividad/{nombre_csv}_act.csv', index=False)


In [25]:
# Cargar los datos
nombres_csv = {1,2,3,4,468,479,4003,4151,4160,4173}

for nombre_csv in nombres_csv:
    data = pd.read_csv(f'Data2_procesados/{nombre_csv}.csv')
    # Calcular y añadir el nivel de actividad (PCA) al DataFrame original `data`
    data_continue = data[['period eating', 'period other', 'period resting', 'period rumination']]
    data['activity_level'] = data_continue.dot(pca_pesos[nombre_csv])


    # Calcular y añadir el nivel de actividad (AFC) al DataFrame original `data`
    data_count = data[['period eating_count', 'period other_count', 'period resting_count', 'period rumination_count']]
    data['activity_level_2'] = data_count.dot(afc_pesos[nombre_csv])

    data.to_csv(f'Data2_actividad/{nombre_csv}_act.csv', index=False)


In [ ]:
# Crear la columna datetime combinando 'Fecha' y 'Hora'
df_7days['FechaHora'] = df_7days.apply(lambda row: pd.Timestamp(row['Fecha']) + pd.Timedelta(hours=row['Hora']), axis=1)

# Asegurarse de que los datos estén ordenados por 'FechaHora'
df_7days.sort_values('FechaHora', inplace=True)

# Visualizar los resultados de PCA y AFC como gráficos de líneas
plt.figure(figsize=(30, 6))
plt.plot(df_7days['FechaHora'], df_7days['activity_level'], marker='o', label='Nivel de Actividad (PCA)')
plt.plot(df_7days['FechaHora'], df_7days['activity_level_2'], marker='x', label='Nivel de Actividad (AFC)')
plt.xlabel('Fecha y Hora')
plt.ylabel('Nivel de Actividad')
plt.title('Nivel de Actividad por Fecha y Hora (PCA y AFC)')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()